In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from collections import defaultdict, Counter
from typing import List, Dict

from tqdm import tqdm
import polars as pl

In [4]:
train = pl.read_csv("/gdrive/MyDrive/amazon_kdd_2023/data/raw/sessions_train.csv")
test = pl.read_csv("/gdrive/MyDrive/amazon_kdd_2023/data/raw/sessions_test_task2.csv")

In [5]:
LOCALES = ["FR", "ES", "IT"]
train = train.filter(pl.col("locale").is_in(LOCALES))
test = test.filter(pl.col("locale").is_in(LOCALES))

In [6]:
# prev_itemsの加工
def str2list(s):
    s = s.replace("[", "").replace("]", "").replace("'", "").replace("\n", " ").replace("\r", " ")
    s = s.split() # 空白で区切ってリスト化
    return s

train = train.with_columns(pl.col("prev_items").apply(str2list).alias("prev_items"))
test = test.with_columns(pl.col("prev_items").apply(str2list).alias("prev_items"))

In [7]:
# session_idの付与
train = train.with_columns(pl.Series(name="session_id", values=["train_" + str(i) for i in range(len(train))]))
test = test.with_columns(pl.Series(name="session_id", values=["test_" + str(i) for i in range(len(test))]))

In [8]:
train.head()

prev_items,next_item,locale,session_id
list[str],str,str,str
"[""B08MV5B53K"", ""B08MV4RCQR"", ""B08MV5B53K""]","""B012408XPC""","""ES""","""train_0"""
"[""B07JGW4QWX"", ""B085VCXHXL""]","""B07JFPYN5P""","""ES""","""train_1"""
"[""B08BFQ52PR"", ""B08LVSTZVF"", ""B08BFQ52PR""]","""B08NJP3KT6""","""ES""","""train_2"""
"[""B08PPBF9C6"", ""B08PPBF9C6"", … ""B08PPBF9C6""]","""B08PP6BLLK""","""ES""","""train_3"""
"[""B0B6W67XCR"", ""B0B712FY2M"", ""B0B6ZYJ3S2""]","""B09SL4MBM2""","""ES""","""train_4"""


In [9]:
test.head()

prev_items,locale,session_id
list[str],str,str
"[""B08GYKNCCP"", ""B08HCPTMJG"", ""B08HCHS64Y""]","""ES""","""test_0"""
"[""B08NYF9MBQ"", ""B085NGXGWM""]","""ES""","""test_1"""
"[""B091FL1QFK"", ""B0B1DG29F4""]","""ES""","""test_2"""
"[""B004APAHCW"", ""B07JMF49HN"", … ""B07JMF49HN""]","""ES""","""test_3"""
"[""B09YM11D4T"", ""B0B12QWP5G"", … ""B07N8N6C85""]","""ES""","""test_4"""


In [10]:
train.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task2/train_task2.parquet")

In [11]:
test.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task2/test_task2.parquet")